In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import sympy

#width，height，列数,行数,中文1英文0,文件序号,子图序号,当前行，当前列
# FX,FY,NC,NR,CN,NN,index,row,col,fontx = [16,10,4,3,1,0,0,0,0,9]     #PPT 多图 display 4*3
# FX,FY,NC,NR,CN,NN,index,row,col,fontx = [19,9.5,6,3,1,0,0,0,0,9]    #PPT display 6*3
FX,FY,NC,NR,CN,NN,index,row,col,fontx = [16,21.33,6,8,0,0,0,0,0,7]    #单图 
EXT,DPI = ['png',300]  # eps svg pdf jpg png
fig,ax =  plt.subplots(figsize=(FX,FY),ncols=NC,nrows=NR,sharex=False,sharey=False)
xxx = np.linspace(0, 1, 100)   #分别代表最小，最大，数量， 生成一个等差数列
indexs = ['01','02','03','04','05','06','07','08','09','10','11','12',
            '13','14','15','16','17','18','19','20','21','22','23','24',
            '25','26','27','28','29','30','31','32','33','34','35','36',
            '37','38','39','40','41','a1','a2','a3','b1','b2','b3','c1']

def fig_clear(cn=CN):
    plt.clf()
    global NN,fig,ax
    fontname = 'SimHei' if cn>0 else 'Arial'
    plt.rcParams['font.sans-serif'] = [fontname]
    plt.rcParams['axes.unicode_minus'] = False
    fig,ax =  plt.subplots(figsize=(FX,FY),ncols=NC,nrows=NR,sharex=True,sharey=True)
    plt.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0)
    NN +=1

def plot_cell(data,lbl,isEnding=0):
    global index, row, col, ax
    index2 = index if index<NC*NR else (index % (NC*NR))
    row = index2 // NC
    col = index2 % NC
    axx = ax[row][col]
    # print(index,index2,row,col)

    e = data['E']
    u = data['U']
    fit = np.polyfit(u,e,2)
    a,b,c = fit
    fitted = xxx**2*a+xxx*b+c
    r = np.corrcoef(u,np.poly1d(fit)(e))[0,1]
    posx0 = 0.3
    posy1 = 0.67 if e.max()<0.5 else 0.32
    posy2 = 0.60 if e.max()<0.5 else 0.25

    minvspan = u.max()
    x = sympy.symbols('x')
    maxv = sympy.solve([x**2*a+x*b+c-fitted.max()],[x])
    maxvspan = 1 if a>0 else maxv[0][0]
    colorvspan = 'tab:red' if minvspan>maxv[0][0] and a<0 else 'tab:green'
    saturationx = maxvspan
    saturationy = fitted.max()
    saturationpx = saturationx+0.02 if saturationx<0.65 else saturationx-0.25
    saturationpy = saturationy+0.04 if saturationy<0.85 else saturationy-0.05

    axx.xaxis.set_major_locator(MultipleLocator(0.2))
    axx.xaxis.set_minor_locator(MultipleLocator(0.1))
    axx.yaxis.set_major_locator(MultipleLocator(0.2))
    axx.yaxis.set_minor_locator(MultipleLocator(0.1))
    axx.tick_params(which='major',direction='in', length=6)
    axx.tick_params(which='minor',direction='in', length=4,width=0.5)
    L1,=axx.plot(u,e,'ko',lw=0.75,label=lbl,markerfacecolor='none')
    L2,=axx.plot(xxx,fitted, color='tab:blue',label='Fitted',lw=2)
    L3,=axx.plot([saturationx],[saturationy],'ro',label='Saturation',lw=20)
    axx.legend(handles=[L1, L2, L3], loc="upper left", ncol=1)
    axx.axvline(x=maxvspan, color='tab:red', linestyle='--',linewidth=1,zorder=100)
    axx.axvspan(xmin=minvspan, xmax=maxvspan, ymin=0, ymax=1,color=colorvspan,alpha=0.15)

    axx.text(0.965,0.96, indexs[index], fontsize='12',fontweight='bold',
                        horizontalalignment='right',verticalalignment='top',
                        transform = axx.transAxes,backgroundcolor='lightgray',
                        fontfamily='sans-serif',zorder=100)
    axx.text(posx0,posy1,r'$E={:.3f}U^2{:+.3f}U{:+.3f}$'.format(fit[0],fit[1],fit[2]), fontsize=fontx)
    axx.text(posx0,posy2,r'$R^2={:.3f}$'.format(r**2), fontsize=fontx)
    axx.text(saturationpx,saturationpy,r'$({:.3f},{:.3f})$'.format(saturationx,saturationy), fontsize=fontx,zorder=999)

    index +=1
    if (row==NR-1 and col==NC-1) or isEnding==1:
        plt.setp(ax[:, :], xlim=[0, 1])
        plt.setp(ax[:, :], ylim=[0, 1])
        plt.setp(ax[-1, :], xlabel='Urbanization')
        plt.setp(ax[:, 0], ylabel='EEQ')
        plt.savefig(fname="E-U-"+str(DPI)+"-"+str(NN)+'.'+EXT,format=EXT, dpi=DPI,pad_inches=0,transparent=True,facecolor='w')
        fig_clear(CN)


## prepare data
E = pd.read_excel('D:/ee-composite/src/sqlserver/data/E/Data_E_1_PC1.xlsx').rename(columns={'pc1':'E'})
U = pd.read_excel('D:/ee-composite/src/sqlserver/data/U/Data_U_1_PC1.xlsx').rename(columns={'pc1':'U'})
df = pd.concat([E,U['U']],axis=1)
fig_clear(CN)

######### Citys
for i in range(1,42,1):
    tmp = df.query('sn=='+str(i))
    lbl = tmp.iat[0,3] if CN>0 else tmp.iat[0,2]
    plot_cell(tmp,lbl)

######### Provinces
for i,item in enumerate(['Anhui','Jiangsu','Zhejiang']):
    tmp = df.query('province=="'+ item +'"').groupby(by='year').mean()
    lbl = item
    plot_cell(tmp,lbl)

######### City types   
for i,item in enumerate(['Magecity','Super city','Large city']):
    tmp = df.query('classid=='+str(i+1)).groupby(by='year').mean()
    plot_cell(tmp,item)

######### study area
tmp = df.groupby(by='year').mean()
plot_cell(tmp,'Study Area',1)
